# IntegratedHTMLPreprocessor 클래스 기능 상세 설명

이 클래스는 HTML 감사보고서 파일을 일괄적으로 전처리하여 구조를 정규화하고, 숫자 및 섹션 정보를 표준화합니다. 아래는 각 기능과 단계별 처리 내역입니다.

## 1. 구조 정규화
- **불필요 태그 제거**: `<script>`, `<style>`, `<noscript>` 등 스킵 태그를 모두 삭제합니다.
- **테이블 구조 정규화**: 클래스가 'table'인 테이블만 처리 대상으로 삼고, 나머지는 무시합니다.
- **텍스트 및 태그 스트림화**: HTML을 순회하며 텍스트와 주요 태그를 선형 스트림으로 변환합니다.
- **빈 태그 및 불필요 공백 정리**: 텍스트 내 불필요한 공백 및 빈 태그를 제거합니다.

## 2. 섹션 및 계층 구조 태깅
- **섹션 태그 래핑**: 주요 섹션(감사보고서, 재무제표, 주석, 감사의견 등)을 `<SEC1>`, `<SEC2>`, `<SEC3>`, `<SEC4>`로 래핑합니다.
- **섹션 제목 자동 추출 및 태깅**: 각 섹션의 제목을 자동으로 추출하여 `<SEC-title>`로 태깅합니다.
- **SEC1 하위 섹션 분리**: SEC1(감사보고서) 내 주요 하위 제목을 기준으로 `<SEC1-1>`, `<SEC1-2>` 등으로 분리합니다.
- **SEC2 테이블 드롭**: SEC2(재무제표) 내 처음 5개의 테이블을 삭제합니다.
- **SEC4 테이블 전체 드롭**: SEC4(감사의견) 내 모든 테이블을 삭제합니다.

## 3. 계층별 텍스트 강조 및 구조화
- **SEC1 하위 제목 강조**: 하위 제목에 `<big>` 태그를 추가하여 강조합니다.
- **SEC3 계층 구조 태깅**: SEC3(주석) 내 대제목, 중제목, 소제목을 각각 `<big>`, `<mid>`, `<small>`로 태깅하고, 계층별로 `<SEC3-x>`, `<SEC3-x.y>`, `<SEC3-x.y.z>`로 래핑합니다.
- **SEC4 주요 제목 강조**: SEC4 내 주요 제목에 `<big>` 태그를 추가합니다.

## 4. 숫자 및 단위 정규화
- **테이블 내 숫자 정규화**:
    - 쉼표 제거: "1,234" → "1234"
    - 괄호 숫자 음수 변환: "(123)" → "-123"
    - 대시(–, —, -)는 "NA"로 변환
- **텍스트 노드 내 숫자 정규화**: 테이블 외부 텍스트에서도 쉼표가 포함된 숫자를 정규화합니다.

## 5. 파일 및 디렉토리 일괄 처리
- **단일 파일 처리**: `process_file()` 메서드로 개별 HTML 파일을 전처리하여 결과를 저장합니다.
- **디렉토리 일괄 처리**: `process_directory()` 메서드로 지정된 폴더 내 모든 HTML 파일을 일괄 처리합니다.
- **출력 파일 저장**: 전처리 결과는 `preprocessed` 폴더에 `_final.html`로 저장됩니다.

## 6. 기타 기능
- **헤더/테일 트리밍**: 불필요한 앞부분과 "외부감사 실시내용" 이후 뒷부분을 자동으로 잘라냅니다.
- **예외 처리 및 로깅**: 처리 중 오류 발생 시 파일별로 실패 내역을 출력합니다.

---
### 전체 처리 흐름
1. HTML 파일 로드 및 파싱
2. 불필요 태그 제거
3. 테이블 및 텍스트 숫자 정규화
4. 선형 스트림 변환 및 구조 정리
5. 섹션 및 계층 구조 태깅
6. 강조 및 래핑 처리
7. 결과 HTML 파일 저장

이 클래스 하나로 감사보고서 HTML 파일의 구조, 숫자, 섹션, 계층, 불필요 요소 제거 등 모든 전처리 과정을 자동화할 수 있습니다.

# 전처리 단계별 설명

1. **구조 정규화**
   - 섹션 태그 추가 (`<section>`, `<subsection>` 등)
   - 계층 구조 명확화
   - 테이블 구조 정규화

2. **숫자 정규화**
   - 쉼표 제거 (예: "1,234" → "1234")
   - 괄호 숫자 음수 변환 (예: "(123)" → "-123")
   - 단위 정규화 (예: "백만원" → "000000")

3. **불필요 요소 제거**
   - 스타일 속성 제거
   - 빈 태그 제거
   - 불필요한 공백 정리

각 단계는 `process_file()` 메서드에서 순차적으로 실행됩니다.

In [2]:
from bs4 import BeautifulSoup, NavigableString, Tag
from pathlib import Path
import html, re, os

class IntegratedHTMLPreprocessor:
    # === 설정 ===
    KEEP_TAGS   = {"table","h1","h2","h3","h4","h5","h6"}
    BLOCK_BREAK = {"p","div","li","ul","ol","section","article","header","footer","aside","address","pre","blockquote","tr"}
    SKIP_TAGS   = {"script","style","noscript"}
    SKIP_CONTAINERS = {"script","style","noscript"}
    SECTION_NAMES = {1:"감사보고서", 2:"재무제표", 3:"주석", 4:"감사의견"}
    SEC1_SUBHEADINGS = {
        "재무제표에 대한 경영진의 책임","감사인의 책임","감사의견","기타사항","핵심감사사항",
        "재무제표에 대한 경영진과 지배기구의 책임","재무제표감사에 대한 감사인의 책임",
    }
    SEC4_HEADINGS = {
        "내부회계관리제도에 대한 감사의견",
        "내부회계관리제도 감사의견근거",
        "내부회계관리제도에 대한 경영진과 지배기구의 책임",
        "내부회계관리제도감사에 대한 감사인의 책임",
        "내부회계관리제도의 정의와 고유한계",
        "내부회계관리제도 검토의견",
    }
    RE_P_CONT_HEADING = re.compile(r"""^\s*["“”]?\d{1,3}(?:\s*[.)])?\s*[^:\n]*?,?\s*계\s*속\s*[:：;]?\s*["“”]?\s*$""", re.X)
    RE_SOLO_CONT_LINE = re.compile(r"""^\s*["“”']?계\s*속\s*[:：;]?\s*["“”']?\s*$""", re.X)
    RE_NUM_ONLY       = re.compile(r"^\s*[\d,]+(?:\.\d+)?%?\s*$")
    RE_MAJOR   = re.compile(r"""^\s*(?P<maj>\d{1,3})\.(?!\d)\s*(?P<title>(?=.*[가-힣A-Za-z]).+?)(?:,?\s*계\s*속)?\s*[:：]?\s*$""", re.X)
    RE_MID_NUM = re.compile(r"""^\s*(?P<maj>\d{1,3})\.\s*(?P<sub>\d{1,3})\s+(?P<title>(?=.*[가-힣A-Za-z]).+?)(?:,?\s*계\s*속)?\s*[:：]?\s*$""", re.X)
    RE_LETTER  = re.compile(r"""^\s*(?:\(|\[)?(?P<enum>[가-힣A-Za-z])(?:\)|\])?\.\s+(?P<title>(?=.*[가-힣A-Za-z]).+?)\s*$""", re.X)
    LETTER_SEQ   = ['가','나','다','라','마','바','사','아','자','차','카','타','파','하']
    LETTER_INDEX = {ch:i for i,ch in enumerate(LETTER_SEQ)}
    RE_NUM_WITH_COMMAS   = re.compile(r"^\s*\d[\d,]*\s*$")
    RE_PARENS_NUM        = re.compile(r"^\s*\(\s*(\d[\d,]*)\s*\)\s*$")
    RE_JUST_DASH         = re.compile(r"^\s*[–—-]\s*$")
    RE_COMMAS_IN_NUMBER  = re.compile(r"(?<=\d),(?=\d)")

    def normalize_ws(self, s: str) -> str:
        s = (s or "").replace("\xa0", " ")
        s = re.sub(r"[ \t]+", " ", s)
        return s

    def _text_from_outer_html(self, v_html: str) -> str:
        try:
            bs = BeautifulSoup(v_html, "html.parser")
            p = bs.find("p")
            if p:
                return self.normalize_ws(p.get_text(strip=True))
            return self.normalize_ws(bs.get_text(strip=True))
        except Exception:
            return ""

    def is_continuation_p(self, text_line: str) -> bool:
        s = self.normalize_ws(text_line).strip().strip('"“”')
        return bool(self.RE_P_CONT_HEADING.match(s))
    def is_solo_continuation(self, text_line: str) -> bool:
        s = self.normalize_ws(text_line).strip()
        return bool(self.RE_SOLO_CONT_LINE.match(s))
    def is_num_only_line(self, s: str) -> bool:
        return bool(self.RE_NUM_ONLY.match(self.normalize_ws(s)))

    def has_section_class(self, tag: Tag) -> bool:
        classes = [str(c).lower() for c in (tag.get("class") or [])]
        return any("section" in c for c in classes)

    def _soup_first_elem(self, html_str: str):
        try:
            return BeautifulSoup(html_str, "html.parser").find(True, recursive=True)
        except Exception:
            return None

    def _has_section_in_outer_html(self, html_str: str) -> bool:
        el = self._soup_first_elem(html_str)
        if not el: return False
        classes = [c.lower() for c in (el.get("class") or [])]
        return any("section" in c for c in classes)

    def _is_table_html(self, s: str) -> bool:
        return isinstance(s, str) and s.lstrip().lower().startswith("<table")

    def traverse_stream(self, node: Tag):
        for child in getattr(node, "children", []):
            if isinstance(child, NavigableString):
                txt = self.normalize_ws(str(child))
                if txt: yield ("text", txt)
                continue
            if not isinstance(child, Tag):
                continue
            name = (child.name or "").lower()
            if name in self.SKIP_TAGS:
                continue
            if name == "br":
                yield ("text", "\n");  continue
            if name in {"h1","h2","h3","h4","h5","h6"} and self.has_section_class(child):
                if not child.get_text(strip=True):
                    continue
            if self.has_section_class(child):
                yield ("keep", str(child));  continue
            if name in self.KEEP_TAGS:
                yield ("keep", str(child));  continue
            yield from self.traverse_stream(child)
            if name in self.BLOCK_BREAK:
                yield ("text", "\n")

    def flatten_to_linear(self, soup: BeautifulSoup):
        body = soup.body or soup
        tokens = list(self.traverse_stream(body))
        linear = []
        buf = ""
        def flush_buf():
            nonlocal buf
            if not buf: return
            for line in buf.split("\n"):
                line = self.normalize_ws(line).strip()
                if not line: continue
                if self.is_continuation_p(line) or self.is_solo_continuation(line):
                    continue
                linear.append(("p", f"<p>{html.escape(line)}</p>"))
            buf = ""
        for kind, payload in tokens:
            if kind == "text":
                buf += payload
            else:
                flush_buf()
                linear.append(("keep", payload))
        flush_buf()
        return linear

    def trim_head_linear(self, linear):
        first_sec_idx = next((i for i,(k,v) in enumerate(linear) if k=="keep" and self._has_section_in_outer_html(v)), None)
        if first_sec_idx is None: return linear
        prev_p_idx = None
        for i in range(first_sec_idx-1, -1, -1):
            if linear[i][0] == "p":
                prev_p_idx = i; break
        start_idx = prev_p_idx if prev_p_idx is not None else first_sec_idx
        return linear[start_idx:]

    def cut_tail_linear(self, linear, stop_phrase="외부감사 실시내용"):
        for i,(k,v) in enumerate(linear):
            if k=="p" and stop_phrase in self._text_from_outer_html(v):
                return linear[:i]
        return linear

    def wrap_sections_linear(self, linear):
        out = []
        sec_idx = 0
        sec_open = False
        for k, v in linear:
            if k=="keep" and self._has_section_in_outer_html(v):
                if sec_open:
                    out.append((None, f"</SEC{sec_idx}>"))
                sec_idx += 1
                sec_name = self.SECTION_NAMES.get(sec_idx, f"섹션{sec_idx}")
                out.append((None, f'<SEC{sec_idx} name="{sec_name}">'))
                sec_open = True
                out.append((k, v))
            else:
                out.append((k, v))
        if sec_open:
            out.append((None, f"</SEC{sec_idx}>"))
        return out

    def split_sec1_subsections_and_drop_tables(self, sec_wrapped):
        open_i = close_i = None
        for i, (k, v) in enumerate(sec_wrapped):
            if k is None and isinstance(v, str) and v.startswith("<SEC1 "):
                open_i = i; break
        if open_i is None: return sec_wrapped
        for j in range(open_i+1, len(sec_wrapped)):
            if sec_wrapped[j][0] is None and isinstance(sec_wrapped[j][1], str) and sec_wrapped[j][1].strip() == "</SEC1>":
                close_i = j; break
        if close_i is None: return sec_wrapped
        before = sec_wrapped[:open_i+1]
        inside = sec_wrapped[open_i+1:close_i]
        after  = sec_wrapped[close_i:]
        out = []
        sub_idx = 0
        sub_open = False
        for k, v in inside:
            if self._is_table_html(v):
                continue
            if k == "p":
                txt = self._text_from_outer_html(v)
                if txt in self.SEC1_SUBHEADINGS:
                    if sub_open:
                        out.append((None, f"</SEC1-{sub_idx}>"))
                    sub_idx += 1
                    out.append((None, f'<SEC1-{sub_idx} name="{html.escape(txt)}">'))
                    sub_open = True
                    out.append((k, v))
                    continue
            out.append((k, v))
        if sub_open:
            out.append((None, f"</SEC1-{sub_idx}>"))
        return before + out + after

    def drop_first_n_tables_in_sec2(self, linear_pairs, n=5):
        open_i = close_i = None
        for i,(k,v) in enumerate(linear_pairs):
            if k is None and isinstance(v,str) and v.startswith("<SEC2 "):
                open_i = i; break
        if open_i is None: return linear_pairs
        for j in range(open_i+1,len(linear_pairs)):
            if linear_pairs[j][0] is None and isinstance(linear_pairs[j][1],str) and linear_pairs[j][1].strip()=="</SEC2>":
                close_i = j; break
        if close_i is None: return linear_pairs
        before = linear_pairs[:open_i+1]
        inside = linear_pairs[open_i+1:close_i]
        after  = linear_pairs[close_i:]
        out = []
        dropped = 0
        for k,v in inside:
            if dropped < n and self._is_table_html(v):
                dropped += 1
                continue
            out.append((k,v))
        return before + out + after

    def drop_all_tables_in_sec4(self, linear_pairs):
        open_i = close_i = None
        for i,(k,v) in enumerate(linear_pairs):
            if k is None and isinstance(v,str) and v.startswith("<SEC4 "):
                open_i = i; break
        if open_i is None: return linear_pairs
        for j in range(open_i+1,len(linear_pairs)):
            if linear_pairs[j][0] is None and isinstance(linear_pairs[j][1],str) and linear_pairs[j][1].strip()=="</SEC4>":
                close_i = j; break
        if close_i is None: return linear_pairs
        before = linear_pairs[:open_i+1]
        inside = linear_pairs[open_i+1:close_i]
        after  = linear_pairs[close_i:]
        out = [(k,v) for (k,v) in inside if not self._is_table_html(v)]
        return before + out + after

    def wrap_sec_titles(self, linear_pairs):
        out = []
        i = 0
        while i < len(linear_pairs):
            k, v = linear_pairs[i]
            out.append((k, v))
            if k is None and isinstance(v, str) and v.startswith("<SEC") and not v.startswith("<SEC-title"):
                j = i + 1
                if j < len(linear_pairs):
                    k2, v2 = linear_pairs[j]
                    if k2 == "keep" and self._has_section_in_outer_html(v2):
                        out.append((None, f"<SEC-title>{v2}</SEC-title>"))
                        i = j
            i += 1
        return out

    def add_big_to_sec1_subheading_ps(self, linear_pairs):
        out = []
        inside_sec1_sub = False
        for k, v in linear_pairs:
            if k is None and isinstance(v, str):
                if v.startswith("<SEC1-"): inside_sec1_sub = True
                elif v.strip().startswith("</SEC1-"): inside_sec1_sub = False
                out.append((k, v)); continue
            if inside_sec1_sub and k == "p":
                txt = self._text_from_outer_html(v)
                if txt in self.SEC1_SUBHEADINGS:
                    out.append((k, f"<big>{v}</big>")); continue
            out.append((k, v))
        return out

    def tag_sec3_headings(self, linear_pairs):
        out = []
        inside_sec3 = False
        parent_state = 'none'
        mid_letter_idx = None
        small_letter_idx = None
        for k, v in linear_pairs:
            if k is None and isinstance(v, str):
                if v.startswith("<SEC3 "):
                    inside_sec3 = True
                    parent_state = 'none'; mid_letter_idx = None; small_letter_idx = None
                    out.append((k, v)); continue
                if v.strip() == "</SEC3>":
                    inside_sec3 = False
                    parent_state = 'none'; mid_letter_idx = None; small_letter_idx = None
                    out.append((k, v)); continue
            if not inside_sec3 or k != "p":
                out.append((k, v)); continue
            text = self._text_from_outer_html(v)
            if not text or self.is_continuation_p(text) or self.is_num_only_line(text):
                out.append((k, v)); continue
            if self.RE_MID_NUM.match(text):
                out.append((k, f"<mid>{v}</mid>"))
                parent_state = 'mid_num'
                small_letter_idx = None
                continue
            if self.RE_MAJOR.match(text):
                out.append((k, f"<big>{v}</big>"))
                parent_state = 'big'
                mid_letter_idx = None; small_letter_idx = None
                continue
            m = self.RE_LETTER.match(text)
            if m:
                ch = m.group("enum")[0]
                idx = self.LETTER_INDEX.get(ch, None)
                if parent_state == 'mid_num':
                    if idx is None:
                        out.append((k, v)); continue
                    if small_letter_idx is None:
                        if idx == 0:
                            out.append((k, f"<small>{v}</small>")); small_letter_idx = 1
                        else:
                            out.append((k, v))
                    else:
                        if idx == small_letter_idx:
                            out.append((k, f"<small>{v}</small>")); small_letter_idx += 1
                        else:
                            out.append((k, v))
                    continue
                if parent_state in ('big','mid_letter'):
                    if idx is None:
                        out.append((k, v)); continue
                    if mid_letter_idx is None:
                        if idx == 0:
                            out.append((k, f"<mid>{v}</mid>"))
                            parent_state = 'mid_letter'; mid_letter_idx = 1
                        else:
                            out.append((k, v))
                    else:
                        if idx == mid_letter_idx:
                            out.append((k, f"<mid>{v}</mid>")); mid_letter_idx += 1
                        else:
                            out.append((k, v))
                    continue
            out.append((k, v))
        return out

    def add_big_to_sec4_heading_ps(self, linear_pairs):
        out = []
        inside_sec4 = False
        for k, v in linear_pairs:
            if k is None and isinstance(v, str):
                if v.startswith("<SEC4 "):
                    inside_sec4 = True
                    out.append((k, v)); continue
                if v.strip() == "</SEC4>":
                    inside_sec4 = False
                    out.append((k, v)); continue
            if inside_sec4 and k == "p":
                txt = self._text_from_outer_html(v)
                if txt in self.SEC4_HEADINGS:
                    out.append((k, f"<big>{v}</big>")); continue
            out.append((k, v))
        return out

    def _find_section_bounds(self, linear_pairs, sec_n: int):
        open_i = close_i = None
        open_tag = f"<SEC{sec_n} "
        close_tag = f"</SEC{sec_n}>"
        for i,(k,v) in enumerate(linear_pairs):
            if k is None and isinstance(v,str) and v.startswith(open_tag):
                open_i = i; break
        if open_i is None:
            return (None, None)
        for j in range(open_i+1, len(linear_pairs)):
            k,v = linear_pairs[j]
            if k is None and isinstance(v,str) and v.strip() == close_tag:
                close_i = j; break
        return (open_i, close_i)

    def nest_sec3_hierarchy(self, linear_pairs):
        out = []
        inside_sec3 = False
        cur_big = None
        cur_mid = None
        cur_small = None
        def close_small():
            nonlocal cur_small
            if cur_small is not None:
                out.append((None, f"</SEC3-{cur_mid}.{cur_small}>"))
                cur_small = None
        def close_mid():
            nonlocal cur_mid, cur_small
            if cur_mid is not None:
                close_small()
                out.append((None, f"</SEC3-{cur_mid}>"))
                cur_mid = None
        def close_big():
            nonlocal cur_big, cur_mid, cur_small
            if cur_big is not None:
                close_mid()
                out.append((None, f"</SEC3-{cur_big}>"))
                cur_big = None
        def open_big(code_str: str):
            nonlocal cur_big
            close_big()
            cur_big = code_str
            out.append((None, f"<SEC3-{cur_big}>"))
        def open_mid(code_str: str):
            nonlocal cur_mid
            close_mid()
            cur_mid = code_str
            out.append((None, f"<SEC3-{cur_mid}>"))
        def open_small(letter: str):
            nonlocal cur_small
            close_small()
            cur_small = letter
            out.append((None, f"<SEC3-{cur_mid}.{cur_small}>"))
        i = 0
        while i < len(linear_pairs):
            k, v = linear_pairs[i]
            if k is None and isinstance(v, str):
                if v.startswith("<SEC3 "):
                    inside_sec3 = True
                    out.append((k, v)); i += 1; continue
                if v.strip() == "</SEC3>":
                    close_big()
                    inside_sec3 = False
                    out.append((k, v)); i += 1; continue
            if not inside_sec3:
                out.append((k, v)); i += 1; continue
            if k == "p":
                s_lower = v.lower()
                is_big   = "<big"   in s_lower
                is_mid   = "<mid"   in s_lower
                is_small = "<small" in s_lower
                if is_big or is_mid or is_small:
                    txt = self._text_from_outer_html(v)
                    if is_big:
                        m = self.RE_MAJOR.match(txt)
                        if m:
                            major = m.group("maj")
                            open_big(major)
                            out.append((k, v)); i += 1; continue
                        out.append((k, v)); i += 1; continue
                    if is_mid:
                        m_num = self.RE_MID_NUM.match(txt)
                        if m_num:
                            code = m_num.group("maj") + "." + m_num.group("sub")
                            open_mid(code)
                            out.append((k, v)); i += 1; continue
                        m_let = self.RE_LETTER.match(txt)
                        if m_let and cur_big is not None:
                            enum = m_let.group("enum")[0]
                            open_mid(f"{cur_big}.{enum}")
                            out.append((k, v)); i += 1; continue
                        out.append((k, v)); i += 1; continue
                    if is_small:
                        m = self.RE_LETTER.match(txt)
                        if m and cur_mid is not None:
                            enum = m.group("enum")[0]
                            open_small(enum)
                            out.append((k, v)); i += 1; continue
                        out.append((k, v)); i += 1; continue
                out.append((k, v)); i += 1; continue
            out.append((k, v)); i += 1; continue
        if inside_sec3:
            close_big()
        return out

    def wrap_section_by_big_linear(self, linear_pairs, sec_n: int):
        open_i, close_i = self._find_section_bounds(linear_pairs, sec_n)
        if open_i is None or close_i is None:
            return linear_pairs
        before = linear_pairs[:open_i+1]
        inside = linear_pairs[open_i+1:close_i]
        after  = linear_pairs[close_i:]
        already_wrapped = any(
            (k is None and isinstance(v,str) and v.lower().startswith(f"<sec{sec_n}-"))
            for k,v in inside
        )
        if already_wrapped:
            return linear_pairs
        big_pos = []
        for idx,(k,v) in enumerate(inside):
            if isinstance(v,str) and v.lstrip().lower().startswith("<big>"):
                big_pos.append(idx)
        if not big_pos:
            return linear_pairs
        out_inside = []
        for b_i, start in enumerate(big_pos, start=1):
            end = big_pos[b_i] if b_i < len(big_pos) else len(inside)
            block = inside[start:end]
            if not block: 
                continue
            out_inside.append((None, f"<SEC{sec_n}-{b_i}>"))
            out_inside.extend(block)
            out_inside.append((None, f"</SEC{sec_n}-{b_i}>"))
        return before + out_inside + after

    # === 숫자 정규화 ===
    def has_class_table(self, tag: Tag) -> bool:
        classes = [(c or "").lower() for c in (tag.get("class") or [])]
        return any(c == "table" or "table" in c for c in classes)

    def get_cell_text(self, cell: Tag) -> str:
        txt = cell.get_text(separator="", strip=True)
        txt = txt.replace("\xa0", " ")
        txt = re.sub(r"[ \t]+", " ", txt)
        return txt.strip()

    def set_cell_text(self, cell: Tag, text: str):
        cell.clear()
        cell.append(text)

    def normalize_table_numbers(self, soup: BeautifulSoup):
        for tbl in soup.find_all("table"):
            if not self.has_class_table(tbl):
                continue
            rows = tbl.find_all("tr", recursive=True)
            for r_idx, tr in enumerate(rows):
                cells = tr.find_all(["th","td"], recursive=False)
                for c_idx, td in enumerate(cells):
                    if r_idx == 0 or c_idx == 0 or td.name.lower() == "th":
                        continue
                    s = self.get_cell_text(td)
                    if not s:
                        continue
                    if self.RE_JUST_DASH.match(s):
                        self.set_cell_text(td, "NA")
                        continue
                    m = self.RE_PARENS_NUM.match(s)
                    if m:
                        num = self.RE_COMMAS_IN_NUMBER.sub("", m.group(1))
                        self.set_cell_text(td, f"-{num}")
                        continue
                    if self.RE_NUM_WITH_COMMAS.match(s):
                        self.set_cell_text(td, self.RE_COMMAS_IN_NUMBER.sub("", s))
                        continue

    def is_inside_class_table_table(self, node: Tag) -> bool:
        t = node.find_parent("table")
        return bool(t and self.has_class_table(t))

    def normalize_text_nodes_commas(self, soup: BeautifulSoup):
        for tag in soup.find_all(True):
            if tag.name in self.SKIP_CONTAINERS:
                continue
            for child in list(tag.children):
                if isinstance(child, NavigableString):
                    if self.is_inside_class_table_table(tag):
                        continue
                    text = str(child)
                    new_text = self.RE_COMMAS_IN_NUMBER.sub("", text)
                    if new_text != text:
                        child.replace_with(new_text)

    def write_linear_to_html(self, linear_pairs, out_path: Path):
        with open(out_path, "w", encoding="utf-8") as f:
            f.write(
                '<!doctype html><meta charset="utf-8"><title>Preprocessed</title>'
                '<style>body{font-family:system-ui,-apple-system,Segoe UI,Roboto,Apple SD Gothic Neo,Malgun Gothic,sans-serif;line-height:1.5}'
                'p{margin:0 0 .7em} table{margin:1em 0;border-collapse:collapse}'
                'td,th{border:1px solid #ddd;padding:.4em}</style><body>\n'
            )
            for k, v in linear_pairs:
                f.write(v + "\n")
            f.write("</body>")
        return out_path

    def process_file(self, in_path: Path, out_dir: Path = None) -> Path:
        html_bytes = in_path.read_bytes()
        soup = BeautifulSoup(html_bytes, "lxml")
        for t in soup.find_all(self.SKIP_TAGS):
            t.decompose()
        self.normalize_table_numbers(soup)
        self.normalize_text_nodes_commas(soup)
        linear = self.flatten_to_linear(soup)
        linear = self.trim_head_linear(linear)
        linear = self.cut_tail_linear(linear, stop_phrase="외부감사 실시내용")
        linear = self.wrap_sections_linear(linear)
        linear = self.split_sec1_subsections_and_drop_tables(linear)
        linear = self.drop_first_n_tables_in_sec2(linear, n=5)
        linear = self.drop_all_tables_in_sec4(linear)
        linear = self.wrap_sec_titles(linear)
        linear = self.add_big_to_sec1_subheading_ps(linear)
        linear = self.tag_sec3_headings(linear)
        linear = self.add_big_to_sec4_heading_ps(linear)
        linear = self.wrap_section_by_big_linear(linear, sec_n=1)
        linear = self.wrap_section_by_big_linear(linear, sec_n=4)
        linear = self.nest_sec3_hierarchy(linear)
        out_dir = out_dir or (in_path.parent / "preprocessed")
        out_dir.mkdir(exist_ok=True)
        out_path = out_dir / f"{in_path.stem}_final.html"
        self.write_linear_to_html(linear, out_path)
        return out_path

    def process_directory(self, dir_path: Path, pattern="*.htm"):
        files = sorted(list(dir_path.glob(pattern)))
        if not files:
            print("No matching files found in", dir_path)
            return
        out_dir = dir_path / "preprocessed"
        out_dir.mkdir(exist_ok=True)
        print(f"Found {len(files)} files. Writing to:", out_dir)
        for fp in files:
            try:
                out_fp = self.process_file(fp, out_dir)
                print("✓ Saved:", out_fp)
            except Exception as e:
                print("✗ Failed:", fp, "->", e)

---

# HtmlAuditToJson 클래스 상세 설명

`HtmlAuditToJson` 클래스는 전처리된 감사보고서 HTML 파일(`*_final.html`)을 받아, 섹션/경로 정보를 보존하면서 문단, 헤딩, 표 단위로 JSON 배열(`list[dict]`)로 변환하는 기능을 제공합니다. 주요 목적은 HTML 구조를 분석하여 각 청크(문단/표/헤딩)를 메타데이터와 함께 구조화된 JSON으로 추출하는 것입니다.

---

## 주요 기능 및 메서드

### 1. 클래스 초기화 및 설정

- **company_default**: 기본 회사명(예: "삼성전자주식회사")
- **version**: 변환 버전 정보(예: "preproc_1.0.0")
- **SECTION_NAMES**: 섹션 번호별 한글 섹션명 매핑

### 2. 텍스트 및 표 유틸리티

- **_norm_text(s)**: 텍스트 내 불필요한 공백 및 특수문자(`\xa0`)를 정규화
- **_cell_text(td)**: 테이블 셀의 텍스트를 정규화하여 추출
- **_serialize_table(tbl)**: 테이블을 문자열로 직렬화(행별로 `|`로 구분), 행/열 개수 shape 정보도 함께 반환

### 3. 섹션 및 제목 추출

- **_nearest_section_context(node)**: HTML 노드 기준으로 가장 가까운 SEC* 섹션 컨테이너의 번호, 이름, 경로를 추출
- **_find_prev_heading_title(node)**: 현재 노드 기준 바로 위쪽 형제에서 제목 후보(`<SEC-title>`, `<big>`, `<mid>`, `<small>` 등)를 탐색하여 반환
- **_classify_type(tag)**: 태그가 표(`table`), 헤딩(`big`/`mid`/`small`), 일반 문단(`p`)인지 판별

### 4. 파일명 기반 정보 추론

- **_infer_company_from_filename(path)**: 파일명에서 회사명 추론(기본값 반환)
- **_infer_year_from_filename(path)**: 파일명에서 연도(YYYY) 추출
- **_infer_title_for_table(tbl)**: 표 바로 위의 헤딩 또는 첫 행의 셀 내용을 기반으로 표 제목 추론

### 5. 핵심 변환 메서드

#### parse_file(html_path: Path) → list[dict]

- 입력: 전처리된 HTML 파일 경로
- 처리:
    - BeautifulSoup으로 HTML 파싱
    - `<p>`, `<table>` 태그를 순회하며 각 청크를 추출
    - 각 청크에 대해 섹션 정보, 제목, 타입, 표 직렬화, 인덱스, 메타데이터(회사, 연도, 파일명 등)를 dict로 생성
    - 모든 청크를 리스트로 반환
- 반환: JSON 직렬화 가능한 dict 리스트

####
 to_json_file(html_path: Path, out_json_path: Path, indent: int = 2) → Path# HtmlAuditToJson 클래스 상세 설명

`HtmlAuditToJson` 클래스는 전처리된 감사보고서 HTML 파일(`*_final.html`)을 받아, 섹션/경로 정보를 보존하면서 문단, 헤딩, 표 단위로 JSON 배열(`list[dict]`)로 변환하는 기능을 제공합니다. 주요 목적은 HTML 구조를 분석하여 각 청크(문단/표/헤딩)를 메타데이터와 함께 구조화된 JSON으로 추출하는 것입니다.

---

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from __future__ import annotations
from bs4 import BeautifulSoup, Tag
from pathlib import Path
import json, re

class HtmlAuditToJson:
    """
    전처리된 감사보고서 *_final.html 한 파일을 받아
    - 섹션/경로(sec_path)를 보존하고
    - 문단/헤딩/표 단위로
    JSON 배열(list[dict])을 만들어 반환하거나 파일로 저장하는 변환기.
    """

    RE_YEAR = re.compile(r"(\d{4})")
    RE_SEC_TAG = re.compile(r"^sec(\d+)(?:[-\.].+)?$", re.I)  # sec1, sec3-1, sec3-1.a
    RE_HAS_TABLE_WORD = re.compile(r"(표|재무상태표|현금흐름표|손익계산서|포괄손익계산서)", re.I)

    SECTION_NAMES = {1: "감사보고서", 2: "재무제표", 3: "주석", 4: "감사의견"}

    def __init__(self, company_default: str = "삼성전자주식회사", version: str = "preproc_1.0.0"):
        self.company_default = company_default
        self.version = version

    # ---------- 유틸 ----------

    @staticmethod
    def _norm_text(s: str | None) -> str:
        if not s: return ""
        s = s.replace("\xa0", " ")
        s = re.sub(r"[ \t]+", " ", s)
        return s.strip()

    def _cell_text(self, td: Tag) -> str:
        return self._norm_text(td.get_text(separator=" ", strip=True))

    def _serialize_table(self, tbl: Tag) -> tuple[str, dict]:
        rows = []
        trs = tbl.find_all("tr", recursive=True)
        max_cols = 0
        for tr in trs:
            cells = tr.find_all(["th", "td"], recursive=False)
            row = [self._cell_text(td) for td in cells]
            max_cols = max(max_cols, len(row))
            rows.append(" | ".join(row))
        txt = "\n".join(rows)
        shape = {"rows": len(rows), "cols": max_cols}
        return txt, shape

    def _nearest_section_context(self, node: Tag) -> tuple[int | None, str | None, str | None]:
        """
        가장 가까운 SEC* 컨테이너 경로/섹션 번호/섹션 이름 찾기
        """
        sec_path = None
        sec_num = None
        sec_name = None
        for anc in [node] + list(node.parents):
            if not isinstance(anc, Tag): continue
            nm = (anc.name or "").lower()
            if nm == "sec-title":
                continue
            m = self.RE_SEC_TAG.match(nm)
            if m:
                sec_path = anc.name.upper()
                sec_num = int(m.group(1))
                if nm == f"sec{sec_num}" and anc.has_attr("name"):
                    sec_name = anc.get("name")
        if sec_num is not None and sec_name is None:
            for anc in list(node.parents):
                if not isinstance(anc, Tag): continue
                if (anc.name or "").lower() == f"sec{sec_num}" and anc.has_attr("name"):
                    sec_name = anc.get("name")
                    break
        return sec_num, sec_name, sec_path

    def _find_prev_heading_title(self, node: Tag) -> str | None:
        """
        위쪽 형제 중 <SEC-title> 또는 <big>/<mid>/<small> 안의 <p> 를 제목으로 사용
        """
        for sib in node.previous_siblings:
            if not isinstance(sib, Tag): continue
            n = (sib.name or "").lower()
            if n == "sec-title":
                t = self._norm_text(sib.get_text(" ", strip=True))
                return t or None
            if n.startswith("sec") and n not in ("sec-title",):
                break  # 섹션 경계 만나면 중단
            if n in ("big", "mid", "small"):
                p = sib.find("p")
                if p:
                    t = self._norm_text(p.get_text(" ", strip=True))
                    if t: return t
            if n == "p" and sib.parent and (sib.parent.name or "").lower() in ("big", "mid", "small"):
                t = self._norm_text(sib.get_text(" ", strip=True))
                if t: return t
        return None

    @staticmethod
    def _classify_type(tag: Tag) -> str:
        if tag.name.lower() == "table":
            return "table"
        if tag.name.lower() == "p":
            parent = tag.parent
            if isinstance(parent, Tag) and (parent.name or "").lower() in ("big", "mid", "small"):
                return "heading"
        return "paragraph"

    def _infer_company_from_filename(self, path: Path) -> str:
        return self.company_default

    def _infer_year_from_filename(self, path: Path) -> int | None:
        m = self.RE_YEAR.search(path.name)
        return int(m.group(1)) if m else None

    def _infer_title_for_table(self, tbl: Tag) -> str | None:
        t = self._find_prev_heading_title(tbl)
        if t: return t
        first_tr = tbl.find("tr")
        if first_tr:
            cells = first_tr.find_all(["th", "td"], recursive=False)
            if cells:
                guess = self._norm_text(" ".join(self._cell_text(c) for c in cells[:2]))
                if self.RE_HAS_TABLE_WORD.search(guess):
                    return guess
        return None

    # ---------- 핵심 변환 ----------

    def parse_file(self, html_path: Path) -> list[dict]:
        """
        입력: 전처리된 *_final.html
        출력: 청크 dict 리스트(JSON 배열로 직렬화 가능)
        """
        soup = BeautifulSoup(html_path.read_bytes(), "lxml")
        body = soup.body or soup

        year = self._infer_year_from_filename(html_path)
        company = self._infer_company_from_filename(html_path)

        counters: dict[str, int] = {}
        chunks: list[dict] = []

        for node in body.find_all(["p", "table"], recursive=True):
            # 빈 <p> 스킵
            if node.name.lower() == "p":
                txt = self._norm_text(node.get_text(" ", strip=True))
                if not txt:
                    continue
            else:
                txt = None  # 표는 직렬화 시 세팅

            section_num, section_name, sec_path = self._nearest_section_context(node)
            if section_num is None:
                continue  # 섹션 바깥 내용은 제외

            key = (sec_path or f"SEC{section_num}").upper()
            chunk_index = counters.get(key, 0)
            counters[key] = chunk_index + 1

            typ = self._classify_type(node)

            # 제목
            if typ == "heading":
                title = txt
            elif typ == "table":
                title = self._infer_title_for_table(node)
            else:
                title = self._find_prev_heading_title(node)

            # 표 직렬화
            is_table = (node.name.lower() == "table")
            table_shape = None
            if is_table:
                txt, table_shape = self._serialize_table(node)

            # ID
            base_company = "SAMSUNG"
            type_marker = "_TABLE" if is_table else ""
            rec_id = f"{key}_{year}_{base_company}{type_marker}_{chunk_index:04d}"

            rec = {
                "id": rec_id,
                "text": txt or "",
                "type": "table" if is_table else ("heading" if typ == "heading" else "paragraph"),
                "section": section_num,
                "section_name": section_name or self.SECTION_NAMES.get(section_num),
                "sec_path": key,
                "title": title,
                "year": year,
                "fiscal_date": None,
                "auditor": None,
                "company": company,
                "source_file": html_path.name,
                "source_path": str(html_path),
                "chunk_index": chunk_index,
                "char_start": None,
                "char_end": None,
                "lang": "ko",
                "is_table": is_table,
                "table_shape": table_shape,
                "keywords": [],
                "tags": [],
                "version": self.version,
            }
            chunks.append(rec)

        return chunks

    def to_json_file(self, html_path: Path, out_json_path: Path, indent: int = 2) -> Path:
        """
        단일 HTML -> JSON 배열 파일로 저장
        """
        chunks = self.parse_file(html_path)
        out_json_path.parent.mkdir(parents=True, exist_ok=True)
        with out_json_path.open("w", encoding="utf-8") as f:
            json.dump(chunks, f, ensure_ascii=False, indent=indent)
        return out_json_path

---

## 실행 파이프라인

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Directory pipeline:
  HTM/HTML → IntegratedHTMLPreprocessor → *_final.html
             → HtmlAuditToJson → *_chunks.json
  and merge all chunks into audit_chunks_all.json (JSON array)

필수: beautifulsoup4, lxml
pip install beautifulsoup4 lxml
"""

from __future__ import annotations
from pathlib import Path
import json, sys, traceback

# === (1) 네가 가진 두 클래스를 import 하거나, 같은 파일에 정의했다면 이 부분 생략 ===
# from your_module_preproc import IntegratedHTMLPreprocessor
# from your_module_json    import HtmlAuditToJson

# --- 여기서는 사용자 메시지에 있던 클래스들을 그대로 사용한다고 가정 ---
# IntegratedHTMLPreprocessor, HtmlAuditToJson 정의가 이미 로드되어 있어야 합니다.

# === 파이프라인 설정 ===
ROOT_DIR = Path("./삼성전자_감사보고서_2014_2024")

# 파일 패턴
PATTERNS = ["*.htm", "*.html"]

# 출력 폴더
PREPROC_DIR_NAME = "preprocessed"      # IntegratedHTMLPreprocessor가 기본으로 씀
JSON_DIR_NAME    = "json"              # JSON 파일 보관

# 합본 파일
MERGED_JSON_PATH = ROOT_DIR / "audit_chunks_all.json"  # JSON 배열(리스트)

def main():
    if not ROOT_DIR.exists():
        print(f"[ERR] Root not found: {ROOT_DIR}", file=sys.stderr)
        sys.exit(1)

    # 수집: *.htm / *.html
    html_files = []
    for pat in PATTERNS:
        html_files.extend(sorted(ROOT_DIR.glob(pat)))

    if not html_files:
        print(f"[ERR] No *.htm(l) files in {ROOT_DIR}", file=sys.stderr)
        sys.exit(1)

    preproc = IntegratedHTMLPreprocessor()
    tojson  = HtmlAuditToJson(company_default="삼성전자주식회사", version="preproc_1.0.0")

    merged_chunks = []
    json_out_dir = ROOT_DIR / JSON_DIR_NAME
    json_out_dir.mkdir(exist_ok=True)

    print(f"[INFO] Found {len(html_files)} raw html files")
    for idx, src in enumerate(html_files, 1):
        try:
            print(f"\n[{idx}/{len(html_files)}] Preprocess: {src.name}")
            # 1) 전처리: *_final.html 생성 (preprocessed 폴더에 저장)
            out_final_path = preproc.process_file(src)  # returns .../preprocessed/<stem>_final.html
            print(f"  -> preprocessed: {out_final_path}")

            # 2) JSON 변환: *_chunks.json (json 폴더에 저장)
            #    원본 이름을 유지해주되 suffix만 교체
            dst_json_path = json_out_dir / (src.stem + "_chunks.json")
            chunks = tojson.parse_file(out_final_path)
            with dst_json_path.open("w", encoding="utf-8") as f:
                json.dump(chunks, f, ensure_ascii=False, indent=2)
            print(f"  -> json saved:   {dst_json_path} (chunks={len(chunks)})")

            # 3) 합본 배열에 추가
            merged_chunks.extend(chunks)

        except Exception as e:
            print(f"[FAIL] {src} -> {e}", file=sys.stderr)
            traceback.print_exc()

    # 4) 전체 합본 저장 (JSON 배열)
    with MERGED_JSON_PATH.open("w", encoding="utf-8") as f:
        json.dump(merged_chunks, f, ensure_ascii=False, indent=2)
    print(f"\n[DONE] Total files: {len(html_files)}, Total chunks: {len(merged_chunks)}")
    print(f"[DONE] Merged JSON: {MERGED_JSON_PATH}")

if __name__ == "__main__":
    main()


[INFO] Found 11 raw html files

[1/11] Preprocess: 감사보고서_2014.htm
  -> preprocessed: 삼성전자_감사보고서_2014_2024/preprocessed/감사보고서_2014_final.html
  -> json saved:   삼성전자_감사보고서_2014_2024/json/감사보고서_2014_chunks.json (chunks=699)

[2/11] Preprocess: 감사보고서_2015.htm
  -> preprocessed: 삼성전자_감사보고서_2014_2024/preprocessed/감사보고서_2015_final.html
  -> json saved:   삼성전자_감사보고서_2014_2024/json/감사보고서_2015_chunks.json (chunks=719)

[3/11] Preprocess: 감사보고서_2016.htm
  -> preprocessed: 삼성전자_감사보고서_2014_2024/preprocessed/감사보고서_2016_final.html
  -> json saved:   삼성전자_감사보고서_2014_2024/json/감사보고서_2016_chunks.json (chunks=777)

[4/11] Preprocess: 감사보고서_2017.htm
  -> preprocessed: 삼성전자_감사보고서_2014_2024/preprocessed/감사보고서_2017_final.html
  -> json saved:   삼성전자_감사보고서_2014_2024/json/감사보고서_2017_chunks.json (chunks=797)

[5/11] Preprocess: 감사보고서_2018.htm
  -> preprocessed: 삼성전자_감사보고서_2014_2024/preprocessed/감사보고서_2018_final.html
  -> json saved:   삼성전자_감사보고서_2014_2024/json/감사보고서_2018_chunks.json (chunks=838)

[6/11] Prepro